In [32]:
import nltk
nltk.download('treebank')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('punkt')
nltk.download('movie_reviews')
nltk.download('stopwords')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Movie Reviews Classifier**

In [0]:
import random
from nltk import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

In [0]:
cats = movie_reviews.categories()
# print(cats)
reviews=[]
for cat in cats: 
  for fid in movie_reviews.fileids(cat):
    review = (list(movie_reviews.words(fid)),cat)
    reviews.append(review)
random.shuffle(reviews)

In [0]:
all_wd_in_reviews = nltk.FreqDist(word.lower() for word in movie_reviews.words())
top_wd_in_reviws = [list(wds) for wds in zip(*all_wd_in_reviews.most_common(2000))][0]
#most_common returns a tuple of words and their freq; 


In [0]:
def ext_ft(review,top_words):
  review_words = set(review)
  ft = {}
  for wd in top_words:
    ft['word_present({})'.format(wd)] = (wd in review_words)
  return ft
# Looks for presence of word of top words and assign 1 if present in review and 0 if not

In [37]:
featureset = [(ext_ft(r,top_wd_in_reviws),c) for (r,c) in reviews]
len(featureset)

2000

In [38]:
print(trainset[:10])

[({'word_present(,)': True, 'word_present(the)': True, 'word_present(.)': True, 'word_present(a)': True, 'word_present(and)': True, 'word_present(of)': True, 'word_present(to)': True, "word_present(')": True, 'word_present(is)': True, 'word_present(in)': True, 'word_present(s)': True, 'word_present(")': True, 'word_present(it)': True, 'word_present(that)': True, 'word_present(-)': True, 'word_present())': True, 'word_present(()': True, 'word_present(as)': True, 'word_present(with)': True, 'word_present(for)': True, 'word_present(his)': True, 'word_present(this)': True, 'word_present(film)': False, 'word_present(i)': True, 'word_present(he)': True, 'word_present(but)': True, 'word_present(on)': True, 'word_present(are)': True, 'word_present(t)': True, 'word_present(by)': True, 'word_present(be)': True, 'word_present(one)': True, 'word_present(movie)': True, 'word_present(an)': True, 'word_present(who)': True, 'word_present(not)': True, 'word_present(you)': True, 'word_present(from)': Tr

In [0]:

trainset,testset = featureset[200:],featureset[:200]

In [40]:
classifier = NaiveBayesClassifier.train(trainset)
print(nltk.classify.accuracy(classifier,testset))

0.82


In [41]:
classifier.show_most_informative_features(20)

Most Informative Features
word_present(outstanding) = True              pos : neg    =     12.7 : 1.0
     word_present(mulan) = True              pos : neg    =      7.7 : 1.0
    word_present(seagal) = True              neg : pos    =      7.7 : 1.0
word_present(wonderfully) = True              pos : neg    =      7.3 : 1.0
     word_present(damon) = True              pos : neg    =      6.2 : 1.0
     word_present(awful) = True              neg : pos    =      5.7 : 1.0
     word_present(flynt) = True              pos : neg    =      5.7 : 1.0
word_present(ridiculous) = True              neg : pos    =      5.3 : 1.0
     word_present(waste) = True              neg : pos    =      5.2 : 1.0
      word_present(lame) = True              neg : pos    =      5.1 : 1.0
    word_present(wasted) = True              neg : pos    =      5.0 : 1.0
    word_present(poorly) = True              neg : pos    =      4.5 : 1.0
     word_present(bland) = True              neg : pos    =      4.4 : 1

In [0]:
dict_vectorizer = None
def get_train_test(train_set,test_set):
  global dict_vectorizer
  dict_vectorizer = DictVectorizer(sparse=False)
  X_train,y_train = zip(*trainset)
  X_train = dict_vectorizer.fit_transform(X_train)
  X_test,y_test=zip(*testset)
  X_test = dict_vectorizer.transform(X_test)
  return X_train,X_test,y_train,y_test
  

In [0]:
X_train,X_test,y_train,y_test = get_train_test(trainset,testset)

In [44]:
rf = RandomForestClassifier(n_estimators=100,n_jobs=4,random_state=10)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [45]:
pred = rf.predict(X_test)
print(accuracy_score(y_test,pred))

0.835


**Repeating same above process with removing stopwords**

In [0]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
all_words_in_reviews =nltk.FreqDist(word.lower() for word in movie_reviews.words() if word not in stopword_list)
top_words_in_reviews =[list(word) for word in zip(*all_words_in_reviews.most_common(2000))][0]

In [0]:
featureset = [(ext_ft(r,top_words_in_reviews),c) for(r,c) in reviews]
train_set,test_set = featureset[200:],featureset[:200]
X_train,X_test,y_train,y_test = get_train_test(train_set,test_set)

In [48]:
rf1 = RandomForestClassifier(n_estimators=100,n_jobs=4,random_state=103)
rf1.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=103, verbose=0,
                       warm_start=False)

In [49]:
pred1 = rf1.predict(X_test)
print(accuracy_score(y_test,pred1))

0.835


In [50]:
featurelist = zip(dict_vectorizer.get_feature_names(),rf.feature_importances_)
featurelist = sorted(featurelist,key = lambda x: x[1], reverse=True)
print(featurelist[:20])

[('word_present(bad)', 0.014827965027999239), ('word_present(worst)', 0.008743858381487675), ('word_present(ridiculous)', 0.0066187248673962055), ('word_present(boring)', 0.006413002630440327), ('word_present(stupid)', 0.006179094516528543), ('word_present(waste)', 0.004122498561799354), ('word_present(wasted)', 0.0038605464279335357), ('word_present(awful)', 0.0035911615410039043), ('word_present(wonderfully)', 0.0035190810707000824), ('word_present(world)', 0.0034811061637786073), ('word_present(wonderful)', 0.0033604126458303433), ('word_present(others)', 0.0032862333984184987), ('word_present(plot)', 0.003228993311282466), ('word_present(lame)', 0.0031233137473653007), ('word_present(supposed)', 0.003109795753537919), ('word_present(script)', 0.003026319720023364), ('word_present(life)', 0.002985078095247025), ('word_present(outstanding)', 0.002889940937893305), ('word_present(brilliant)', 0.002883122510930932), ('word_present(mess)', 0.002836343677623522)]
